# Proportional Approval Voting implementation
Todo: use counts of unique cvrs, not all cvrs

In [234]:
import pandas as pd
import collections
import itertools
import time

In [91]:
def harmonic(matches):
    "Calculate harmonic series sum up to the integer 'matches'"
    utility = 0.0
    for i in range(1, matches + 1):
        utility += 1 / i

    return utility

In [112]:
UTILITY = [harmonic(i) for i in range(0, 100)]

In [113]:
UTILITY

[0.0,
 1.0,
 1.5,
 1.8333333333333333,
 2.083333333333333,
 2.283333333333333,
 2.4499999999999997,
 2.5928571428571425,
 2.7178571428571425,
 2.8289682539682537,
 2.9289682539682538,
 3.0198773448773446,
 3.103210678210678,
 3.180133755133755,
 3.251562326562327,
 3.3182289932289937,
 3.3807289932289937,
 3.439552522640758,
 3.4951080781963135,
 3.547739657143682,
 3.597739657143682,
 3.6453587047627294,
 3.690813250217275,
 3.73429151108684,
 3.7759581777535067,
 3.8159581777535068,
 3.854419716215045,
 3.8914567532520823,
 3.927171038966368,
 3.9616537975870574,
 3.9949871309203906,
 4.02724519543652,
 4.05849519543652,
 4.08879822573955,
 4.118209990445433,
 4.146781419016861,
 4.174559196794639,
 4.201586223821666,
 4.22790201329535,
 4.2535430389363755,
 4.278543038936376,
 4.302933282838815,
 4.326742806648339,
 4.349998620601827,
 4.3727258933290996,
 4.394948115551322,
 4.416687245986104,
 4.4379638417307845,
 4.4587971750641175,
 4.4792053383294235,
 4.499205338329423,
 4.518

In [110]:
def utility(permutation, cvr):
    """Return utility to voter who voted cvr of given permutation
    >>> utility(frozenset(["c1", "c2", "c4"]), frozenset(["c1", "c3", "c4"]))
    1.5
    """
    return UTILITY[len(permutation.intersection(cvr))]

In [114]:
import doctest
doctest.testmod()

TestResults(failed=0, attempted=1)

In [2]:
df = pd.read_csv("/srv/voting/audit/corla/adams-2015/Byers_SD_32J.csv")

In [3]:
df

,Choice_564_1,Choice_567_1,Choice_570_1,Choice_573_1,Choice_576_1,Choice_579_1,Choice_582_1,Choice_585_1
0,1,0,0,1,0,1,0,1
1,0,1,1,1,1,0,0,0
2,0,1,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0
4,0,1,1,1,0,0,0,0
5,1,1,0,0,1,1,0,0
6,0,1,0,1,1,1,0,0
7,1,1,0,1,0,1,0,0
8,0,1,0,1,0,1,0,1
9,1,1,0,1,1,0,0,0


In [60]:
cvrs = [frozenset(key for t in df.itertuples(False)   for key, value in t._asdict().items() if value)]

In [61]:
cvrs

[frozenset({'Choice_564_1',
            'Choice_567_1',
            'Choice_570_1',
            'Choice_573_1',
            'Choice_576_1',
            'Choice_579_1',
            'Choice_582_1',
            'Choice_585_1'})]

In [68]:
c = []

In [118]:
next(df.itertuples(False))

Pandas(Choice_564_1=1, Choice_567_1=0, Choice_570_1=0, Choice_573_1=1, Choice_576_1=0, Choice_579_1=1, Choice_582_1=0, Choice_585_1=1)

In [121]:
all_candidates = type(next(df.itertuples(False)))._fields

In [122]:
all_candidates

('Choice_564_1',
 'Choice_567_1',
 'Choice_570_1',
 'Choice_573_1',
 'Choice_576_1',
 'Choice_579_1',
 'Choice_582_1',
 'Choice_585_1')

In [70]:
for t in df.itertuples(False):
    c.append(frozenset(key for key, value in t._asdict().items() if value))
             

In [71]:
c

[frozenset({'Choice_564_1', 'Choice_573_1', 'Choice_579_1', 'Choice_585_1'}),
 frozenset({'Choice_567_1', 'Choice_570_1', 'Choice_573_1', 'Choice_576_1'}),
 frozenset({'Choice_567_1', 'Choice_573_1'}),
 frozenset({'Choice_573_1'}),
 frozenset({'Choice_567_1', 'Choice_570_1', 'Choice_573_1'}),
 frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_576_1', 'Choice_579_1'}),
 frozenset({'Choice_567_1', 'Choice_573_1', 'Choice_576_1', 'Choice_579_1'}),
 frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_573_1', 'Choice_579_1'}),
 frozenset({'Choice_567_1', 'Choice_573_1', 'Choice_579_1', 'Choice_585_1'}),
 frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_573_1', 'Choice_576_1'}),
 frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_573_1', 'Choice_576_1'}),
 frozenset({'Choice_567_1', 'Choice_570_1', 'Choice_573_1', 'Choice_585_1'}),
 frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_573_1', 'Choice_585_1'}),
 frozenset({'Choice_570_1', 'Choice_573_1', 'Choice_579_1', 'Choice_585_1'}),
 fro

In [72]:
len(c)

121

In [73]:
cc = collections.Counter(c)

In [74]:
len(cc)

50

In [75]:
cc

Counter({frozenset({'Choice_573_1'}): 3,
         frozenset({'Choice_564_1',
                    'Choice_567_1',
                    'Choice_573_1',
                    'Choice_579_1'}): 3,
         frozenset({'Choice_570_1'}): 1,
         frozenset({'Choice_570_1',
                    'Choice_573_1',
                    'Choice_579_1',
                    'Choice_585_1'}): 2,
         frozenset({'Choice_567_1', 'Choice_570_1', 'Choice_573_1'}): 1,
         frozenset({'Choice_564_1'}): 5,
         frozenset({'Choice_564_1',
                    'Choice_570_1',
                    'Choice_582_1',
                    'Choice_585_1'}): 3,
         frozenset({'Choice_564_1',
                    'Choice_567_1',
                    'Choice_573_1',
                    'Choice_576_1'}): 10,
         frozenset({'Choice_582_1'}): 1,
         frozenset({'Choice_564_1',
                    'Choice_567_1',
                    'Choice_573_1',
                    'Choice_582_1'}): 2,
         frozense

In [116]:
v1

frozenset({'Choice_564_1', 'Choice_573_1', 'Choice_579_1', 'Choice_585_1'})

In [117]:
type(v1)

frozenset

In [168]:
len(c)

121

In [183]:
len(all_candidatesdidates)

8

In [251]:
timings = []

In [199]:
possible_results = itertools.combinations(all_candidates, 4)

In [233]:
# next(possible_results)

# about 12 seconds(?) for 20 candidates, 10 winners, 121 
Memory pressure with 24 and 12, nearly 3 GB used
24 choose 12 = 2704156

In [282]:
candidates = 30
winners = 5

In [283]:
possible_results = itertools.combinations(all_candidates + tuple(list(range(candidates - len(all_candidates)))), winners)

In [284]:
t1 = time.time()
scores = {}
for result in possible_results:
    resultset = frozenset(result)
    scores[resultset] = sum(utility(resultset, cvr) for cvr in c)
timings.append("{} seconds for {} candidates, {} winners, {} ballots".format(time.time() - t1, candidates, winners, 121))

In [285]:
timings

['0.005549907684326172 seconds for 8 candidates, 4 winners, 121 ballots',
 '0.0004897117614746094 seconds for 8 candidates, 4 winners, 121 ballots',
 '0.022046566009521484 seconds for 10 candidates, 5 winners, 121 ballots',
 '0.22809982299804688 seconds for 14 candidates, 7 winners, 121 ballots',
 '2.5786209106445312 seconds for 18 candidates, 9 winners, 121 ballots',
 '9.772108793258667 seconds for 20 candidates, 10 winners, 121 ballots',
 '39.48445129394531 seconds for 22 candidates, 11 winners, 121 ballots',
 '161.45553970336914 seconds for 24 candidates, 12 winners, 121 ballots',
 '4.725412607192993 seconds for 24 candidates, 5 winners, 121 ballots',
 '6.555292844772339 seconds for 30 candidates, 5 winners, 121 ballots']

In [229]:
winner = max(scores.items(), key=lambda tuple: tuple[1])

In [230]:
print("Max score: {} for {}".format(scores[winner[0]], winner))

Max score: 234.00000000000037 for (frozenset({0, 'Choice_573_1', 'Choice_564_1', 'Choice_585_1', 10, 'Choice_570_1', 'Choice_582_1', 'Choice_576_1', 'Choice_567_1', 'Choice_579_1'}), 234.00000000000037)


In [195]:
resultset

frozenset({'Choice_576_1', 'Choice_579_1', 'Choice_582_1', 'Choice_585_1'})

In [203]:
utility(resultset, c[0])

1.5

In [202]:
len(scores)

70

In [201]:
sorted(scores.items(), key=lambda tuple: tuple[1], reverse=True)

[(frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_570_1', 'Choice_573_1'}),
  182.33333333333343),
 (frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_570_1', 'Choice_576_1'}),
  182.3333333333334),
 (frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_570_1', 'Choice_585_1'}),
  180.83333333333337),
 (frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_576_1', 'Choice_585_1'}),
  179.5833333333334),
 (frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_573_1', 'Choice_576_1'}),
  179.33333333333343),
 (frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_573_1', 'Choice_585_1'}),
  177.58333333333348),
 (frozenset({'Choice_564_1', 'Choice_570_1', 'Choice_573_1', 'Choice_576_1'}),
  177.50000000000003),
 (frozenset({'Choice_564_1', 'Choice_567_1', 'Choice_570_1', 'Choice_582_1'}),
  174.83333333333337),
 (frozenset({'Choice_564_1', 'Choice_573_1', 'Choice_576_1', 'Choice_585_1'}),
  174.5833333333334),
 (frozenset({'Choice_564_1', 'Choice_570_1', 'Choice_573_1', 'Choice_585_1'}),
  173.

In [159]:
scores

{frozenset({'Choice_564_1',
            'Choice_567_1',
            'Choice_573_1',
            'Choice_579_1'}): 164.41666666666674,
 frozenset({'Choice_564_1',
            'Choice_570_1',
            'Choice_573_1',
            'Choice_579_1'}): 161.5,
 frozenset({'Choice_564_1',
            'Choice_567_1',
            'Choice_570_1',
            'Choice_579_1'}): 170.00000000000003,
 frozenset({'Choice_570_1',
            'Choice_573_1',
            'Choice_579_1',
            'Choice_585_1'}): 138.16666666666666,
 frozenset({'Choice_570_1',
            'Choice_573_1',
            'Choice_576_1',
            'Choice_579_1'}): 149.49999999999997,
 frozenset({'Choice_567_1',
            'Choice_570_1',
            'Choice_573_1',
            'Choice_582_1'}): 162.83333333333331,
 frozenset({'Choice_564_1',
            'Choice_570_1',
            'Choice_582_1',
            'Choice_585_1'}): 161.91666666666669,
 frozenset({'Choice_567_1',
            'Choice_573_1',
            'Choice

In [76]:
v1 = c[0]

In [77]:
v2 = c[2]

In [95]:
v1.intersection(v2)

frozenset({'Choice_573_1'})

In [79]:
len(v1.intersection(v2))

1

In [4]:
df.iloc[0]

Choice_564_1    1
Choice_567_1    0
Choice_570_1    0
Choice_573_1    1
Choice_576_1    0
Choice_579_1    1
Choice_582_1    0
Choice_585_1    1
Name: 0, dtype: int64

In [39]:
[t for t in df.itertuples(False)]

[Pandas(Choice_564_1=1, Choice_567_1=0, Choice_570_1=0, Choice_573_1=1, Choice_576_1=0, Choice_579_1=1, Choice_582_1=0, Choice_585_1=1),
 Pandas(Choice_564_1=0, Choice_567_1=1, Choice_570_1=1, Choice_573_1=1, Choice_576_1=1, Choice_579_1=0, Choice_582_1=0, Choice_585_1=0),
 Pandas(Choice_564_1=0, Choice_567_1=1, Choice_570_1=0, Choice_573_1=1, Choice_576_1=0, Choice_579_1=0, Choice_582_1=0, Choice_585_1=0),
 Pandas(Choice_564_1=0, Choice_567_1=0, Choice_570_1=0, Choice_573_1=1, Choice_576_1=0, Choice_579_1=0, Choice_582_1=0, Choice_585_1=0),
 Pandas(Choice_564_1=0, Choice_567_1=1, Choice_570_1=1, Choice_573_1=1, Choice_576_1=0, Choice_579_1=0, Choice_582_1=0, Choice_585_1=0),
 Pandas(Choice_564_1=1, Choice_567_1=1, Choice_570_1=0, Choice_573_1=0, Choice_576_1=1, Choice_579_1=1, Choice_582_1=0, Choice_585_1=0),
 Pandas(Choice_564_1=0, Choice_567_1=1, Choice_570_1=0, Choice_573_1=1, Choice_576_1=1, Choice_579_1=1, Choice_582_1=0, Choice_585_1=0),
 Pandas(Choice_564_1=1, Choice_567_1=1, C

In [50]:
t._asdict()

OrderedDict([('Choice_564_1', 1),
             ('Choice_567_1', 0),
             ('Choice_570_1', 0),
             ('Choice_573_1', 1),
             ('Choice_576_1', 0),
             ('Choice_579_1', 1),
             ('Choice_582_1', 0),
             ('Choice_585_1', 1)])

In [57]:
frozenset(key for key, value in t._asdict().items() if value)

frozenset({'Choice_564_1', 'Choice_573_1', 'Choice_579_1', 'Choice_585_1'})

In [52]:
d = t._asdict()

In [54]:
d.items()

ItemsView(OrderedDict([('Choice_564_1', 1), ('Choice_567_1', 0), ('Choice_570_1', 0), ('Choice_573_1', 1), ('Choice_576_1', 0), ('Choice_579_1', 1), ('Choice_582_1', 0), ('Choice_585_1', 1)]))

In [40]:
t = _[0]

In [48]:
t.count(1)

4

In [9]:
v = df.to_records()

In [12]:
v[:3]

rec.array([(0, 1, 0, 0, 1, 0, 1, 0, 1), (1, 0, 1, 1, 1, 1, 0, 0, 0),
 (2, 0, 1, 0, 1, 0, 0, 0, 0)], 
          dtype=[('index', '<i8'), ('Choice_564_1', '<i8'), ('Choice_567_1', '<i8'), ('Choice_570_1', '<i8'), ('Choice_573_1', '<i8'), ('Choice_576_1', '<i8'), ('Choice_579_1', '<i8'), ('Choice_582_1', '<i8'), ('Choice_585_1', '<i8')])

In [30]:
set_counts = collections.Counter(tuple(row[1:]) for row in map(tuple, v))

In [31]:
len(set_counts)

50

In [32]:
set_counts

Counter({(0, 0, 0, 0, 0, 0, 0, 1): 1,
         (0, 0, 0, 0, 0, 0, 1, 0): 1,
         (0, 0, 0, 0, 1, 0, 0, 0): 1,
         (0, 0, 0, 1, 0, 0, 0, 0): 3,
         (0, 0, 1, 0, 0, 0, 0, 0): 1,
         (0, 0, 1, 0, 0, 0, 0, 1): 2,
         (0, 0, 1, 0, 0, 1, 0, 1): 1,
         (0, 0, 1, 0, 1, 0, 1, 1): 5,
         (0, 0, 1, 0, 1, 1, 0, 1): 1,
         (0, 0, 1, 0, 1, 1, 1, 0): 2,
         (0, 0, 1, 1, 0, 0, 1, 1): 2,
         (0, 0, 1, 1, 0, 1, 0, 1): 2,
         (0, 0, 1, 1, 1, 0, 1, 0): 1,
         (0, 1, 0, 0, 1, 0, 1, 1): 2,
         (0, 1, 0, 1, 0, 0, 0, 0): 1,
         (0, 1, 0, 1, 0, 0, 1, 1): 2,
         (0, 1, 0, 1, 0, 1, 0, 1): 4,
         (0, 1, 0, 1, 1, 0, 0, 1): 3,
         (0, 1, 0, 1, 1, 1, 0, 0): 1,
         (0, 1, 1, 0, 0, 0, 1, 1): 1,
         (0, 1, 1, 1, 0, 0, 0, 0): 1,
         (0, 1, 1, 1, 0, 0, 0, 1): 1,
         (0, 1, 1, 1, 1, 0, 0, 0): 1,
         (1, 0, 0, 0, 0, 0, 0, 0): 5,
         (1, 0, 0, 0, 1, 1, 0, 1): 1,
         (1, 0, 0, 1, 0, 1, 0, 1): 1,
         (1,

In [25]:
len(vs)

121

In [20]:
vs = [frozenset(row) for row in map(tuple, v)]

In [21]:
vs

[frozenset({0, 1}),
 frozenset({0, 1}),
 frozenset({0, 1, 2}),
 frozenset({0, 1, 3}),
 frozenset({0, 1, 4}),
 frozenset({0, 1, 5}),
 frozenset({0, 1, 6}),
 frozenset({0, 1, 7}),
 frozenset({0, 1, 8}),
 frozenset({0, 1, 9}),
 frozenset({0, 1, 10}),
 frozenset({0, 1, 11}),
 frozenset({0, 1, 12}),
 frozenset({0, 1, 13}),
 frozenset({0, 1, 14}),
 frozenset({0, 1, 15}),
 frozenset({0, 1, 16}),
 frozenset({0, 1, 17}),
 frozenset({0, 1, 18}),
 frozenset({0, 1, 19}),
 frozenset({0, 1, 20}),
 frozenset({0, 1, 21}),
 frozenset({0, 1, 22}),
 frozenset({0, 1, 23}),
 frozenset({0, 1, 24}),
 frozenset({0, 1, 25}),
 frozenset({0, 1, 26}),
 frozenset({0, 1, 27}),
 frozenset({0, 1, 28}),
 frozenset({0, 1, 29}),
 frozenset({0, 1, 30}),
 frozenset({0, 1, 31}),
 frozenset({0, 1, 32}),
 frozenset({0, 1, 33}),
 frozenset({0, 1, 34}),
 frozenset({0, 1, 35}),
 frozenset({0, 1, 36}),
 frozenset({0, 1, 37}),
 frozenset({0, 1, 38}),
 frozenset({0, 1, 39}),
 frozenset({0, 1, 40}),
 frozenset({0, 1, 41}),
 frozens

In [18]:
tuple(map(tuple, v))

((0, 1, 0, 0, 1, 0, 1, 0, 1),
 (1, 0, 1, 1, 1, 1, 0, 0, 0),
 (2, 0, 1, 0, 1, 0, 0, 0, 0),
 (3, 0, 0, 0, 1, 0, 0, 0, 0),
 (4, 0, 1, 1, 1, 0, 0, 0, 0),
 (5, 1, 1, 0, 0, 1, 1, 0, 0),
 (6, 0, 1, 0, 1, 1, 1, 0, 0),
 (7, 1, 1, 0, 1, 0, 1, 0, 0),
 (8, 0, 1, 0, 1, 0, 1, 0, 1),
 (9, 1, 1, 0, 1, 1, 0, 0, 0),
 (10, 1, 1, 0, 1, 1, 0, 0, 0),
 (11, 0, 1, 1, 1, 0, 0, 0, 1),
 (12, 1, 1, 0, 1, 0, 0, 0, 1),
 (13, 0, 0, 1, 1, 0, 1, 0, 1),
 (14, 1, 0, 0, 0, 0, 0, 0, 0),
 (15, 1, 0, 0, 0, 0, 0, 0, 0),
 (16, 1, 1, 1, 0, 0, 0, 0, 1),
 (17, 1, 0, 1, 0, 0, 0, 1, 1),
 (18, 1, 1, 0, 0, 1, 0, 0, 1),
 (19, 1, 0, 0, 0, 1, 1, 0, 1),
 (20, 1, 0, 1, 0, 0, 0, 1, 1),
 (21, 0, 1, 0, 1, 0, 1, 0, 1),
 (22, 0, 1, 0, 1, 0, 1, 0, 1),
 (23, 1, 1, 0, 1, 0, 0, 1, 0),
 (24, 0, 0, 1, 1, 0, 1, 0, 1),
 (25, 1, 1, 0, 1, 1, 0, 0, 0),
 (26, 1, 1, 1, 1, 0, 0, 0, 0),
 (27, 1, 1, 1, 0, 0, 0, 0, 1),
 (28, 0, 0, 1, 0, 1, 0, 1, 1),
 (29, 0, 0, 1, 0, 1, 0, 1, 1),
 (30, 1, 1, 0, 0, 1, 0, 0, 0),
 (31, 1, 1, 1, 1, 0, 0, 0, 0),
 (32, 1, 0, 1, 0, 

In [5]:
df.apply(lambda x:x)

,Choice_564_1,Choice_567_1,Choice_570_1,Choice_573_1,Choice_576_1,Choice_579_1,Choice_582_1,Choice_585_1
0,1,0,0,1,0,1,0,1
1,0,1,1,1,1,0,0,0
2,0,1,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0
4,0,1,1,1,0,0,0,0
5,1,1,0,0,1,1,0,0
6,0,1,0,1,1,1,0,0
7,1,1,0,1,0,1,0,0
8,0,1,0,1,0,1,0,1
9,1,1,0,1,1,0,0,0
